In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import torch
import random
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [22]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,LENGTH,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,180.0,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,180.0,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,370.0,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,120.0,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,300.0,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391934,TRAIN_391934,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,...,110.0,KEJZ24,"China, People's Republic Of",NaN,NaN,NaN,NaN,14,0.000552,0.000000
391935,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,80.0,JLTM64,Vietnam,-0.66,0.97,27.3,1.253491,8,0.002615,144.061389
391936,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,200.0,YERJ68,Singapore,-3.44,7.99,21.1,4.766257,18,0.000155,41.482222
391937,TRAIN_391937,TW,JWI3,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,...,170.0,HCZK58,Comoros,NaN,NaN,NaN,NaN,15,0.000990,7.485278


In [23]:
# 1. Load the USD data
usd_data = pd.read_csv("USD.csv")

# 2. Convert the '날짜' column in USD data to datetime format
#usd_data['날짜'] = pd.to_datetime(usd_data['날짜'], format='%Y- %m- %d').dt.date
usd_data['날짜'] = pd.to_datetime(usd_data['날짜'], format='%Y-%m-%d').dt.date

# Convert the 'ATA' column in train and test data to only the date format
train['ATA_date'] = pd.to_datetime(train['ATA']).dt.date
test['ATA_date'] = pd.to_datetime(test['ATA']).dt.date

# 3. Merge the USD data with train and test data on the date columns
train_merged = pd.merge(train, usd_data, left_on='ATA_date', right_on='날짜', how='left')
test_merged = pd.merge(test, usd_data, left_on='ATA_date', right_on='날짜', how='left')

# Drop the '날짜' and 'ATA_date' columns from merged datasets as they are redundant
train_merged.drop(['날짜', 'ATA_date'], axis=1, inplace=True)
test_merged.drop(['날짜', 'ATA_date'], axis=1, inplace=True)
train_merged.drop(['거래량', '변동 %'],axis=1,inplace=True)
test_merged.drop(['거래량', '변동 %'],axis=1,inplace=True)
train_merged.drop(['시가', '고가', '저가'],axis=1,inplace=True)
test_merged.drop(['시가', '고가', '저가'],axis=1,inplace=True)
train_merged['종가'] = train_merged['종가'].str.replace(',', '').astype(float)
test_merged['종가'] = test_merged['종가'].str.replace(',', '').astype(float)
train = train_merged.copy()
test = test_merged.copy()
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR,종가
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000,1129.55
1,TRAIN_000001,IN,UJM2,Bulk,0.000000,2014-09-23 6:59,X886125,30.0,13,35900,...,SPNO34,Marshall Islands,NaN,NaN,NaN,NaN,12,0.000217,0.000000,1040.15
2,TRAIN_000002,CN,EUC8,Container,0.000000,2015-02-03 22:00,T674582,50.0,12,146000,...,FNPK22,Malta,NaN,NaN,NaN,NaN,6,0.001614,0.000000,1093.04
3,TRAIN_000003,JP,ZAG4,Container,0.000000,2020-01-17 4:02,Y847238,20.0,18,6910,...,PBZV77,Bahamas,-3.18,-1.61,6.7,2.629350,13,0.000356,0.000000,1160.44
4,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444,1169.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391934,TRAIN_391934,JP,QYY1,Container,0.000000,2017-06-06 5:02,Y375615,20.0,27,6820,...,KEJZ24,"China, People's Republic Of",NaN,NaN,NaN,NaN,14,0.000552,0.000000,1118.08
391935,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,JLTM64,Vietnam,-0.66,0.97,27.3,1.253491,8,0.002615,144.061389,1185.51
391936,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,YERJ68,Singapore,-3.44,7.99,21.1,4.766257,18,0.000155,41.482222,1132.90
391937,TRAIN_391937,TW,JWI3,Container,9.448179,2015-01-08 7:15,J644215,30.0,29,23800,...,HCZK58,Comoros,NaN,NaN,NaN,NaN,15,0.000990,7.485278,1095.51


# 0포함 o x

In [24]:
train = train[train['DIST'] != 0]
train = train.reset_index(drop = True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,SHIPMANAGER,FLAG,U_WIND,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR,종가
0,TRAIN_000000,SG,GIW5,Container,30.881018,2018-12-17 21:29,Z618338,30.0,24,24300,...,CQSB78,Panama,NaN,NaN,NaN,NaN,5,0.002615,3.450000,1129.55
1,TRAIN_000004,SG,GIW5,Container,27.037650,2020-01-26 7:51,A872328,50.0,10,116000,...,GUCE76,Liberia,-0.33,-3.28,25.6,2.495953,15,0.002615,253.554444,1169.20
2,TRAIN_000005,AU,WHH4,Bulk,49.953585,2021-03-05 18:36,S731836,40.0,7,183000,...,HZUO14,Japan,6.10,-2.84,28.1,4.016217,5,0.000103,68.391389,1128.00
3,TRAIN_000006,ID,REJ1,Container,42.276281,2016-12-11 3:00,A735263,20.0,30,6800,...,HCOS27,Indonesia,NaN,NaN,NaN,NaN,10,0.000041,31.700556,1172.89
4,TRAIN_000009,CN,NGG6,Container,101.521598,2018-11-30 19:29,S458225,50.0,7,124000,...,YLMR26,United Kingdom,NaN,NaN,NaN,NaN,3,0.001743,58.193056,1120.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234641,TRAIN_391932,CN,WAF5,Bulk,21.866691,2017-06-01 15:27,V427658,40.0,13,169000,...,TNIW71,Marshall Islands,NaN,NaN,NaN,NaN,23,0.000618,89.884167,1122.13
234642,TRAIN_391933,CN,NCU8,Bulk,9.647703,2020-03-02 13:37,H321245,30.0,26,69000,...,PPMJ71,Panama,3.64,3.44,21.4,3.298553,21,0.000939,1095.597222,1189.61
234643,TRAIN_391935,SG,GIW5,Bulk,5.884603,2019-10-16 0:36,K635567,10.0,12,3160,...,JLTM64,Vietnam,-0.66,0.97,27.3,1.253491,8,0.002615,144.061389,1185.51
234644,TRAIN_391936,US,QGN3,Bulk,70.660241,2021-03-23 22:35,J284147,30.0,8,60300,...,YERJ68,Singapore,-3.44,7.99,21.1,4.766257,18,0.000155,41.482222,1132.90


In [25]:
train.drop(['SAMPLE_ID'],axis=1,inplace=True)
test.drop(['SAMPLE_ID'],axis=1,inplace=True)

In [26]:
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    #df['hour'] = df['ATA'].dt.hour
    #df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

In [27]:
# Categorical 컬럼 인코딩
from sklearn.preprocessing import LabelEncoder

categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']


for feature in tqdm(categorical_features, desc="Encoding features"):
    encoder = LabelEncoder()
    train[feature] = encoder.fit_transform(train[feature])
    for label in np.unique(test[feature]):
        if label not in encoder.classes_:
            encoder.classes_ = np.append(encoder.classes_, label)
    test[feature] = encoder.transform(test[feature])

# 결측치 처리
train.fillna(train.mean(), inplace=True)
test.fillna(train.mean(), inplace=True)

Encoding features:   0%|          | 0/6 [00:00<?, ?it/s]

Encoding features: 100%|██████████| 6/6 [00:11<00:00,  1.89s/it]


# 0

In [28]:
label = 'CI_HOUR'
eval_metric = 'mean_absolute_error'

In [10]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)

#predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['RF','GBM','XGB'])
predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='best_quality',  ag_args_fit={'num_gpus': 0})

y_pred_best = predictor.predict(test_data)
y_pred_best = pd.DataFrame(y_pred_best, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred_best['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231018_173455\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231018_173455\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   23.33 GB / 498.62 GB (4.7%)
Train Data Rows:    234646
Train Data Columns: 27
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.002777778, 1

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-0.003345,-104.361655,900.693840,904.163923,0.099090,900.693840,904.163923,0.099090,1,True,2
1,RandomForestMSE_BAG_L1,-46.410799,-90.403649,1.916742,7.082437,114.085697,1.916742,7.082437,114.085697,1,True,3
2,WeightedEnsemble_L2,-46.423226,-89.479478,904.158989,917.576113,147.015628,0.006005,0.003003,1.406278,2,True,5
3,ExtraTreesMSE_BAG_L2,-50.161845,-89.555028,1807.495845,1964.450499,184.816986,1.406278,6.381801,38.993442,2,True,7
4,WeightedEnsemble_L3,-51.891205,-89.510544,1809.282470,1971.891262,353.305130,0.010009,0.003002,0.966879,3,True,8
5,ExtraTreesMSE_BAG_L1,-52.427497,-89.954507,1.542402,6.326750,31.424563,1.542402,6.326750,31.424563,1,True,4
6,RandomForestMSE_BAG_L2,-62.415460,-90.541928,1807.866182,1965.506459,313.344810,1.776615,7.437761,167.521265,2,True,6
7,KNeighborsUnif_BAG_L1,-84.415406,-104.464650,901.936583,1040.495588,0.214195,901.936583,1040.495588,0.214195,1,True,1


In [11]:
predictor.feature_importance(train, silent=True)

KeyboardInterrupt: 

In [53]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 220491 entries, 0 to 220490
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ARI_CO              220491 non-null  int32  
 1   ARI_PO              220491 non-null  int32  
 2   SHIP_TYPE_CATEGORY  220491 non-null  int32  
 3   DIST                220491 non-null  float64
 4   ID                  220491 non-null  int32  
 5   BREADTH             220491 non-null  float64
 6   BUILT               220491 non-null  int64  
 7   DEADWEIGHT          220491 non-null  int64  
 8   DEPTH               220491 non-null  float64
 9   DRAUGHT             220491 non-null  float64
 10  GT                  220491 non-null  int64  
 11  LENGTH              220491 non-null  float64
 12  SHIPMANAGER         220491 non-null  int32  
 13  FLAG                220491 non-null  int32  
 14  U_WIND              220491 non-null  float64
 15  V_WIND              220491 non-nul

In [9]:
from autogluon.tabular import TabularDataset, TabularPredictor

# 각 ARI_CO 별로 데이터 분할 후 학습 및 예측
unique_ari_co = train['year'].unique()

for ari_co in unique_ari_co:
    # ARI_CO 별 데이터 분할
    train_subset = train[train['year'] == ari_co].copy()
    test_subset = test[test['year'] == ari_co].copy()
    #train_subset.drop(['year'],axis=1,inplace=True)
    #test_subset.drop(['year'],axis=1,inplace=True)    

    # 데이터셋 변환
    train_data = TabularDataset(train_subset)
    test_data = TabularDataset(test_subset)
    
    # 모델 학습
    predictor = TabularPredictor(label='CI_HOUR', eval_metric='mean_absolute_error', path = None).fit(
        train_data, 
        presets='best_quality',
        #ag_args_fit={'num_gpus': 0},
        #ag_args_fit={'num_cpus': 1, 'num_gpus': 0}
        #ag_args_fit={'num_gpus': 0, 'num_cpus': 1}
        num_gpus=1
        #included_model_types=['CAT']
    )
    
    # 예측 및 결과 저장
    y_pred = predictor.predict(test_data)
    submission.loc[test_subset.index, 'CI_HOUR'] = y_pred.values
    print(ari_co)
    print(ari_co)
    print(ari_co)
    print(ari_co)
    print(ari_co)
    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    print(len(train_subset))
    predictor.leaderboard(train, silent=True)    

# 최종 결과 확인
print(submission)
submission

No path specified. Models will be saved in: "AutogluonModels\ag-20231019_145255\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231019_145255\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   192.00 GB / 511.09 GB (37.6%)
Train Data Rows:    28005
Train Data Columns: 25
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2151.602778, 0.014166667, 93.25425, 194.93553)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicla

2018
2018
2018
2018
2018
28005
28005
28005
28005
28005


No path specified. Models will be saved in: "AutogluonModels\ag-20231019_150303\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231019_150303\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   185.80 GB / 511.09 GB (36.4%)
Train Data Rows:    26852
Train Data Columns: 25
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2152.193889, 0.016388889, 104.38929, 213.1725)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicla

2020
2020
2020
2020
2020
26852
26852
26852
26852
26852


No path specified. Models will be saved in: "AutogluonModels\ag-20231019_151332\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231019_151332\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   183.36 GB / 511.09 GB (35.9%)
Train Data Rows:    38810
Train Data Columns: 25
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2157.861389, 0.100833333, 106.47821, 211.1771)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multicla

...

No path specified. Models will be saved in: "AutogluonModels\ag-20231019_161351\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231019_161351\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   168.74 GB / 511.09 GB (33.0%)
Train Data Rows:    5694
Train Data Columns: 25
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1186.935278, 0.635833333, 76.99137, 121.86614)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclas

2023
2023
2023
2023
2023
5694
5694
5694
5694
5694


No path specified. Models will be saved in: "AutogluonModels\ag-20231019_161745\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231019_161745\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   168.12 GB / 511.09 GB (32.9%)
Train Data Rows:    2138
Train Data Columns: 25
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2040.676667, 0.0725, 213.46147, 324.2631)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', '

2014
2014
2014
2014
2014
2138
2138
2138
2138
2138
          SAMPLE_ID    CI_HOUR
0       TEST_000000  30.150824
1       TEST_000001  28.454451
2       TEST_000002  26.296280
3       TEST_000003  86.548218
4       TEST_000004   0.329801
...             ...        ...
220486  TEST_220486  92.346962
220487  TEST_220487  60.992531
220488  TEST_220488  26.162733
220489  TEST_220489   5.040003
220490  TEST_220490  34.408936

[220491 rows x 2 columns]


,SAMPLE_ID,CI_HOUR
0,TEST_000000,30.150824
1,TEST_000001,28.454451
2,TEST_000002,26.296280
3,TEST_000003,86.548218
4,TEST_000004,0.329801
...,...,...
220486,TEST_220486,92.346962
220487,TEST_220487,60.992531
220488,TEST_220488,26.162733
220489,TEST_220489,5.040003


In [30]:
train

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,CI_HOUR,종가,year,month,day,weekday
0,17,21,2,30.881018,21975,30.0,24,24300,10.0,10.0,...,19.295802,2.658560,5,0.002615,3.450000,1129.55,2018,12,17,0
1,17,21,2,27.037650,807,50.0,10,116000,20.0,10.0,...,25.600000,2.495953,15,0.002615,253.554444,1169.20,2020,1,26,6
2,0,91,0,49.953585,16077,40.0,7,183000,20.0,20.0,...,28.100000,4.016217,5,0.000103,68.391389,1128.00,2021,3,5,4
3,6,67,2,42.276281,650,20.0,30,6800,10.0,10.0,...,19.295802,2.658560,10,0.000041,31.700556,1172.89,2016,12,11,6
4,4,45,2,101.521598,15818,50.0,7,124000,30.0,20.0,...,19.295802,2.658560,3,0.001743,58.193056,1120.44,2018,11,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234641,4,89,0,21.866691,18428,40.0,13,169000,20.0,20.0,...,19.295802,2.658560,23,0.000618,89.884167,1122.13,2017,6,1,3
234642,4,44,0,9.647703,6146,30.0,26,69000,20.0,10.0,...,21.400000,3.298553,21,0.000939,1095.597222,1189.61,2020,3,2,0
234643,17,21,0,5.884603,9087,10.0,12,3160,10.0,10.0,...,27.300000,1.253491,8,0.002615,144.061389,1185.51,2019,10,16,2
234644,21,61,0,70.660241,7861,30.0,8,60300,20.0,10.0,...,21.100000,4.766257,18,0.000155,41.482222,1132.90,2021,3,23,1


In [29]:
test

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,PORT_SIZE,종가,year,month,day,weekday
0,17,21,2,1.826589,8718,50.0,18,117000,30.0,20.0,...,1.63000,27.100000,1.587063,19,0.002615,1212.50,2020,6,18,3
1,4,90,1,25.399386,3731,10.0,13,3810,10.0,10.0,...,-2.33000,14.200000,2.663972,6,0.001028,1116.61,2021,5,26,2
2,4,45,2,111.079467,4962,20.0,26,10900,10.0,10.0,...,-4.91000,9.300000,3.255315,8,0.001743,1167.48,2019,12,16,0
3,2,19,0,9.175258,653,30.0,9,55800,20.0,10.0,...,-0.06336,19.295802,2.658560,2,0.000182,1169.64,2015,11,16,0
4,8,66,2,0.000000,750,30.0,19,39800,20.0,10.0,...,-0.06336,19.295802,2.658560,10,0.000552,1139.86,2018,10,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220486,1,76,0,21.712733,4073,60.0,15,298000,30.0,20.0,...,-0.06336,19.295802,2.658560,13,0.000080,1125.10,2017,4,4,1
220487,2,23,0,4.870490,14092,30.0,9,80500,20.0,10.0,...,-0.06336,19.295802,2.658560,13,0.000039,1123.74,2016,8,22,0
220488,8,88,3,17.068286,17213,10.0,14,1200,0.0,0.0,...,1.22000,27.400000,2.405268,23,0.000264,1294.68,2022,7,10,6
220489,4,62,1,0.000000,1869,10.0,27,3420,10.0,0.0,...,-0.64000,8.200000,2.199039,22,0.000595,1095.60,2020,12,28,0


In [31]:
train.to_csv('SAINT_TRAIN.csv',index=False)
test.to_csv('SAINT_TEST.csv',index=False)

In [11]:
test = pd.read_csv('test.csv')
all = pd.concat([test,submission],axis=1)
#all['CI_HOUR'][all['DIST'] == 0] = 0
all.loc[all['DIST'] == 0, 'CI_HOUR'] = 0
submission['CI_HOUR'] = all['CI_HOUR']
#submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('SEP_NEW_EXCLUDE_0.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,30.150824
1,TEST_000001,28.454451
2,TEST_000002,26.296280
3,TEST_000003,86.548218
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,92.346962
220487,TEST_220487,60.992531
220488,TEST_220488,26.162733
220489,TEST_220489,0.000000


In [33]:
z = train[['CI_HOUR']].copy()
z = z[z['CI_HOUR'] != 0]
z.describe()

,CI_HOUR
count,234646.000000
mean,103.450118
std,210.795773
min,0.002778
25%,12.703125
50%,36.002917
75%,99.740347
max,2159.130556


In [30]:
a = pd.read_csv('NEW_VERSION1.csv')
a = a[a['CI_HOUR'] != 0]
a.describe()

,CI_HOUR
count,146652.000000
mean,102.612986
std,204.702926
min,1.425004
25%,17.598591
50%,40.946835
75%,95.859177
max,2065.305176


In [29]:
b = pd.read_csv('NEW_V_P.csv')
b = b[b['CI_HOUR'] != 0]
b.describe()

,CI_HOUR
count,132061.000000
mean,64.002258
std,98.202330
min,0.217417
25%,20.696900
50%,39.623740
75%,76.239235
max,1542.976800


In [39]:
new_v_p_data = pd.read_csv('NEW_V_P.csv')

min_value = new_v_p_data['CI_HOUR'].min()
max_value = new_v_p_data['CI_HOUR'].max()
scaled_ci_hour = new_v_p_data.copy()
scaled_ci_hour['CI_HOUR'] = (new_v_p_data['CI_HOUR'] - min_value) / (max_value - min_value) * 2000
scaled_ci_hour['CI_HOUR'] = (new_v_p_data['CI_HOUR']*0.5 + scaled_ci_hour['CI_HOUR']*0.5)
scaled_ci_hour.to_csv('testing2.csv',index=False)
scaled_ci_hour

,SAMPLE_ID,CI_HOUR
0,TEST_000000,4.201626
1,TEST_000001,17.887573
2,TEST_000002,30.305592
3,TEST_000003,101.903278
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,62.115261
220487,TEST_220487,54.566376
220488,TEST_220488,28.786565
220489,TEST_220489,0.000000


In [65]:
a = pd.read_csv('NEW_V_P.csv')
sub = submission.copy()
sub['CI_HOUR'] = (a['CI_HOUR']*0.5 + submission['CI_HOUR']*0.5)
sub.to_csv('jugwang.csv',index=False)
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,31.217644
1,TEST_000001,22.565344
2,TEST_000002,25.581632
3,TEST_000003,89.963154
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,72.539057
220487,TEST_220487,52.043848
220488,TEST_220488,27.433267
220489,TEST_220489,0.000000


# CAT

In [124]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy as np


def objective(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
            'iterations':trial.suggest_int("iterations", 300, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.1, 1),
            'depth': trial.suggest_int('depth',5, 16),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'reg_lambda': trial.suggest_uniform('reg_lambda',30,100),
            'subsample': trial.suggest_uniform('subsample',0.3,1),
            'random_strength': trial.suggest_uniform('random_strength',10,100),
            'od_wait':trial.suggest_int('od_wait', 10, 150),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,20),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 1, 100),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0., 1.0),
            'random_state' : 724,
            'verbose' : 0,
        }
    #'task_type' : 'GPU',
    #"eval_metric":'RMSE',
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val,y_val)],cat_features=cat_features,
              verbose=False)
    cat_pred = cat.predict(X_val)
    score = mean_absolute_error(y_val, cat_pred)
    return score


In [40]:
from sklearn.preprocessing import LabelEncoder
import bisect
'''
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']

# 결측치 처리
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(train_x.mean(), inplace=True)
'''
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG', 'country_cluster', 'PORT_SIZE_Zone', 'BREADTH', 'DEPTH', 'DRAUGHT']

# 수치형 변수만 대상으로 결측치 대체
numeric_cols = train_x.select_dtypes(include=[np.number]).columns

# 훈련 데이터의 평균 계산
mean_values = train_x[numeric_cols].mean()

# 결측치 대체
train_x[numeric_cols] = train_x[numeric_cols].fillna(mean_values)
test[numeric_cols] = test[numeric_cols].fillna(mean_values)

In [126]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220055 entries, 0 to 220054
Data columns (total 57 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ARI_CO                 220055 non-null  object 
 1   ARI_PO                 220055 non-null  object 
 2   SHIP_TYPE_CATEGORY     220055 non-null  object 
 3   DIST                   220055 non-null  float64
 4   ID                     220055 non-null  object 
 5   BREADTH                220055 non-null  object 
 6   BUILT                  220055 non-null  int64  
 7   DEADWEIGHT             220055 non-null  int64  
 8   DEPTH                  220055 non-null  object 
 9   DRAUGHT                220055 non-null  object 
 10  GT                     220055 non-null  int64  
 11  LENGTH                 220055 non-null  float64
 12  SHIPMANAGER            220055 non-null  object 
 13  FLAG                   220055 non-null  object 
 14  U_WIND                 220055 non-nu

In [127]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)


In [128]:
kf = KFold(n_splits=5, shuffle=True, random_state=724)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=724))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best MAE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['CI_HOUR'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-05 23:13:14,949] A new study created in memory with name: no-name-d5bdd458-3bc8-4f77-9659-7cde3598a38d


Optimizing hyperparameters for fold 1...


[I 2023-10-05 23:15:17,832] Trial 0 finished with value: 55.43213467773245 and parameters: {'iterations': 921, 'learning_rate': 0.7313877507251003, 'depth': 10, 'min_data_in_leaf': 15, 'reg_lambda': 30.77074089604671, 'subsample': 0.5286843392696118, 'random_strength': 54.372999344451735, 'od_wait': 51, 'leaf_estimation_iterations': 18, 'bagging_temperature': 1.0913808956698392, 'colsample_bylevel': 0.36753066317430727}. Best is trial 0 with value: 55.43213467773245.
[I 2023-10-05 23:17:27,695] Trial 1 finished with value: 46.73698461577896 and parameters: {'iterations': 433, 'learning_rate': 0.5899605678803301, 'depth': 9, 'min_data_in_leaf': 15, 'reg_lambda': 62.223289044475145, 'subsample': 0.9759050658077331, 'random_strength': 30.55733714009755, 'od_wait': 125, 'leaf_estimation_iterations': 20, 'bagging_temperature': 10.099229114607697, 'colsample_bylevel': 0.8290967984097537}. Best is trial 1 with value: 46.73698461577896.
[I 2023-10-05 23:19:36,021] Trial 2 finished with value: 

Best RMSE for fold 1: 42.25324087037929
Best hyperparameters for fold 1: {'iterations': 792, 'learning_rate': 0.4094644098911695, 'depth': 8, 'min_data_in_leaf': 13, 'reg_lambda': 43.85829584203627, 'subsample': 0.6745303236399682, 'random_strength': 85.6116646925902, 'od_wait': 62, 'leaf_estimation_iterations': 16, 'bagging_temperature': 51.306767359876446, 'colsample_bylevel': 0.4699924477380927}
0:	learn: 210.0952407	total: 96.6ms	remaining: 1m 16s
1:	learn: 210.0940552	total: 147ms	remaining: 58.1s
2:	learn: 210.0938004	total: 208ms	remaining: 54.6s
3:	learn: 209.1358980	total: 362ms	remaining: 1m 11s
4:	learn: 207.6132118	total: 492ms	remaining: 1m 17s
5:	learn: 206.2839777	total: 653ms	remaining: 1m 25s
6:	learn: 206.0279903	total: 791ms	remaining: 1m 28s
7:	learn: 204.0681873	total: 966ms	remaining: 1m 34s
8:	learn: 204.0638360	total: 1.01s	remaining: 1m 28s
9:	learn: 203.4280451	total: 1.22s	remaining: 1m 35s
10:	learn: 203.1901894	total: 1.39s	remaining: 1m 38s
11:	learn: 203.

[I 2023-10-06 00:47:49,969] A new study created in memory with name: no-name-34d199d9-2296-4ac7-aab4-4affaab2fe81


...

Best RMSE for fold 4: 42.99479569369386
Best hyperparameters for fold 4: {'iterations': 792, 'learning_rate': 0.4094644098911695, 'depth': 8, 'min_data_in_leaf': 13, 'reg_lambda': 43.85829584203627, 'subsample': 0.6745303236399682, 'random_strength': 85.6116646925902, 'od_wait': 62, 'leaf_estimation_iterations': 16, 'bagging_temperature': 51.306767359876446, 'colsample_bylevel': 0.4699924477380927}
0:	learn: 207.5462183	total: 148ms	remaining: 1m 56s
1:	learn: 206.2793843	total: 235ms	remaining: 1m 32s
2:	learn: 206.2774985	total: 287ms	remaining: 1m 15s
3:	learn: 204.7584634	total: 371ms	remaining: 1m 13s
4:	learn: 204.1595926	total: 541ms	remaining: 1m 25s
5:	learn: 204.1418121	total: 604ms	remaining: 1m 19s
6:	learn: 204.1093265	total: 700ms	remaining: 1m 18s
7:	learn: 202.6380865	total: 845ms	remaining: 1m 22s
8:	learn: 202.1132612	total: 1.02s	remaining: 1m 28s
9:	learn: 201.9492653	total: 1.1s	remaining: 1m 26s
10:	learn: 201.9431037	total: 1.16s	remaining: 1m 22s
11:	learn: 201.

[I 2023-10-06 05:35:54,142] A new study created in memory with name: no-name-9a45990b-9b73-4ff6-9786-80073f391832


Optimizing hyperparameters for fold 5...


[I 2023-10-06 05:38:13,591] Trial 0 finished with value: 57.20639981749408 and parameters: {'iterations': 921, 'learning_rate': 0.7313877507251003, 'depth': 10, 'min_data_in_leaf': 15, 'reg_lambda': 30.77074089604671, 'subsample': 0.5286843392696118, 'random_strength': 54.372999344451735, 'od_wait': 51, 'leaf_estimation_iterations': 18, 'bagging_temperature': 1.0913808956698392, 'colsample_bylevel': 0.36753066317430727}. Best is trial 0 with value: 57.20639981749408.
[I 2023-10-06 05:40:26,699] Trial 1 finished with value: 46.48582358805089 and parameters: {'iterations': 433, 'learning_rate': 0.5899605678803301, 'depth': 9, 'min_data_in_leaf': 15, 'reg_lambda': 62.223289044475145, 'subsample': 0.9759050658077331, 'random_strength': 30.55733714009755, 'od_wait': 125, 'leaf_estimation_iterations': 20, 'bagging_temperature': 10.099229114607697, 'colsample_bylevel': 0.8290967984097537}. Best is trial 1 with value: 46.48582358805089.
[I 2023-10-06 05:42:35,209] Trial 2 finished with value: 

Best RMSE for fold 5: 43.48887650381214
Best hyperparameters for fold 5: {'iterations': 783, 'learning_rate': 0.2584880573013857, 'depth': 12, 'min_data_in_leaf': 30, 'reg_lambda': 45.02231140060744, 'subsample': 0.6753500090427036, 'random_strength': 21.788829786258407, 'od_wait': 90, 'leaf_estimation_iterations': 16, 'bagging_temperature': 58.67357912276298, 'colsample_bylevel': 0.5379457040607726}
0:	learn: 205.2582218	total: 697ms	remaining: 9m 5s
1:	learn: 204.2043019	total: 886ms	remaining: 5m 45s
2:	learn: 203.0624090	total: 1.01s	remaining: 4m 23s
3:	learn: 202.0806996	total: 1.23s	remaining: 3m 59s
4:	learn: 200.1313413	total: 1.92s	remaining: 4m 59s
5:	learn: 198.2753633	total: 2.61s	remaining: 5m 37s
6:	learn: 197.5619923	total: 3.39s	remaining: 6m 16s
7:	learn: 197.5466068	total: 3.45s	remaining: 5m 34s
8:	learn: 197.1022517	total: 3.64s	remaining: 5m 13s
9:	learn: 196.9068203	total: 3.75s	remaining: 4m 50s
10:	learn: 196.8625524	total: 3.9s	remaining: 4m 33s
11:	learn: 195

In [129]:
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('CAT_v2_724.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,85.724445
1,TEST_000001,290.797561
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,57.593767
...,...,...
244984,TEST_244984,69.481320
244985,TEST_244985,418.025254
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


In [58]:
train

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,nBase,SpAbs_A,...,MACCS_key_158,MACCS_key_159,MACCS_key_160,MACCS_key_161,MACCS_key_162,MACCS_key_163,MACCS_key_164,MACCS_key_165,MLM,HLM
0,3.259,400.495,5,2,8,3.259,117.37,0,0,35.689316,...,1,1,1,1,1,1,1,1,26.010,50.680
1,2.169,301.407,2,1,2,2.172,73.47,0,0,26.575899,...,1,0,1,1,1,1,1,1,29.270,50.590
2,1.593,297.358,5,0,3,1.585,62.45,2,1,29.802128,...,1,0,1,1,1,1,0,1,5.586,80.892
3,4.771,494.652,6,0,5,3.475,92.60,0,1,45.884166,...,1,1,1,1,1,1,1,1,5.710,2.000
4,2.335,268.310,3,0,1,2.337,42.43,0,0,26.308663,...,1,1,1,1,1,1,1,1,93.270,99.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,3.409,396.195,3,1,5,3.409,64.74,0,0,30.902711,...,1,0,1,1,1,0,1,1,1.556,3.079
3467,1.912,359.381,4,1,3,1.844,77.37,0,0,35.887372,...,1,1,1,1,1,1,1,1,35.560,47.630
3468,1.941,261.320,3,1,6,2.124,70.14,0,0,23.546531,...,1,1,1,1,1,1,1,1,56.150,1.790
3469,0.989,284.696,5,1,5,0.989,91.51,0,0,23.936088,...,1,1,0,1,1,1,1,1,0.030,2.770


# TabNet

In [50]:
import optuna
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# Objective 함수 정의
def objective(trial, X_train, y_train, X_val, y_val):
    n_d = trial.suggest_int("n_d", 4, 64)
    n_a = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 1, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    optimizer_params = {"lr": lr}
    
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_params=optimizer_params
    )
    
    y_train_2d = y_train.values.reshape(-1, 1)
    y_val_2d = y_val.values.reshape(-1, 1)
    
    model.fit(
        X_train=X_train.values, 
        y_train=y_train_2d,
        eval_set=[(X_val.values, y_val_2d)],
        eval_metric=['mae'], 
        max_epochs=50, 
        patience=10
    )
    
    preds_val = model.predict(X_val.values)
    mae = mean_absolute_error(y_val.values.ravel(), preds_val)
    return mae

In [45]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)
# object 타입 피처만 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
test_unknown = test.copy()

for col in train_x.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    train_x[col] = le.fit_transform(train_x[col])
    
    known_mask = test[col].isin(le.classes_)
    test_unknown = test_unknown[~known_mask]  # 새로운 레이블을 포함하는 데이터 포인트를 별도로 저장
    test = test[known_mask]
    
    test[col] = le.transform(test[col])
    label_encoders[col] = le

In [47]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [51]:
kf = KFold(n_splits=5, shuffle=True, random_state=724)
fold_results = []
test_predictions = []

for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=724))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best MAE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = TabNetRegressor(**best_params)
    model.fit(X_train.values, y_train.values.ravel(), max_epochs=100)
    test_pred = model.predict(test.values)
    test_predictions.append(test_pred)

final_prediction = np.mean(test_predictions, axis=0)

for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

[I 2023-10-08 01:57:59,512] A new study created in memory with name: no-name-a4896c90-e6f1-42a6-8c8b-5140d4f7cb5f


Optimizing hyperparameters for fold 1...
epoch 0  | loss: 54667.76901| val_0_mae: 180.92095|  0:00:12s
epoch 1  | loss: 54281.59527| val_0_mae: 96.3394 |  0:00:23s
epoch 2  | loss: 53814.79642| val_0_mae: 95.458  |  0:00:35s
epoch 3  | loss: 53561.90072| val_0_mae: 94.85518|  0:00:47s
epoch 4  | loss: 53108.98061| val_0_mae: 93.88412|  0:00:59s
epoch 5  | loss: 52801.55621| val_0_mae: 93.0834 |  0:01:10s
epoch 6  | loss: 52390.89816| val_0_mae: 92.26418|  0:01:21s
epoch 7  | loss: 52056.07431| val_0_mae: 91.56768|  0:01:32s
epoch 8  | loss: 51702.23693| val_0_mae: 91.00958|  0:01:43s
epoch 9  | loss: 51476.37852| val_0_mae: 90.44323|  0:01:54s
epoch 10 | loss: 51061.01298| val_0_mae: 89.9735 |  0:02:06s
epoch 11 | loss: 50774.96551| val_0_mae: 89.50916|  0:02:18s
epoch 12 | loss: 50536.75444| val_0_mae: 89.09913|  0:02:29s
epoch 13 | loss: 50189.4527| val_0_mae: 88.73736|  0:02:41s
epoch 14 | loss: 49885.91626| val_0_mae: 88.31281|  0:02:52s
epoch 15 | loss: 49535.19113| val_0_mae: 88.

[I 2023-10-08 02:04:09,494] Trial 0 finished with value: 87.4515860027796 and parameters: {'n_d': 58, 'n_a': 46, 'n_steps': 5, 'gamma': 1.471768994831443, 'lambda_sparse': 1.0520131499846483e-05, 'lr': 9.551591485457036e-05}. Best is trial 0 with value: 87.4515860027796.


epoch 0  | loss: 55200.8212| val_0_mae: 140.91934|  0:00:18s
epoch 1  | loss: 54819.6298| val_0_mae: 104.29145|  0:00:35s
epoch 2  | loss: 54864.47222| val_0_mae: 99.48548|  0:00:54s
epoch 3  | loss: 54627.99508| val_0_mae: 98.81957|  0:01:13s
epoch 4  | loss: 54556.0473| val_0_mae: 98.27959|  0:01:32s
epoch 5  | loss: 54470.91333| val_0_mae: 97.81953|  0:01:51s
epoch 6  | loss: 54251.61036| val_0_mae: 97.32818|  0:02:10s
epoch 7  | loss: 54020.38877| val_0_mae: 96.90192|  0:02:29s
epoch 8  | loss: 53893.73273| val_0_mae: 96.4771 |  0:02:48s
epoch 9  | loss: 53832.84439| val_0_mae: 96.04991|  0:03:07s
epoch 10 | loss: 53692.07525| val_0_mae: 95.65555|  0:03:27s
epoch 11 | loss: 53605.63461| val_0_mae: 95.25395|  0:03:47s
epoch 12 | loss: 53461.50434| val_0_mae: 94.9103 |  0:04:06s
epoch 13 | loss: 53462.24061| val_0_mae: 94.5333 |  0:04:24s
epoch 14 | loss: 53168.84983| val_0_mae: 94.22555|  0:04:46s
epoch 15 | loss: 53158.89853| val_0_mae: 93.87284|  0:05:05s
epoch 16 | loss: 52942.24

[I 2023-10-08 02:21:06,939] Trial 1 finished with value: 87.50547043557528 and parameters: {'n_d': 34, 'n_a': 21, 'n_steps': 9, 'gamma': 1.0189881736525601, 'lambda_sparse': 5.433270488189108e-05, 'lr': 3.736575233292654e-05}. Best is trial 0 with value: 87.4515860027796.


...

[I 2023-10-08 02:38:26,739] Trial 3 finished with value: 87.22914795206387 and parameters: {'n_d': 53, 'n_a': 62, 'n_steps': 6, 'gamma': 1.8290967984097537, 'lambda_sparse': 2.0638315057558745e-05, 'lr': 0.00014753325905081117}. Best is trial 3 with value: 87.22914795206387.


epoch 0  | loss: 55057.72515| val_0_mae: 103.21543|  0:00:19s
epoch 1  | loss: 54999.30688| val_0_mae: 97.88925|  0:00:39s
epoch 2  | loss: 54923.15769| val_0_mae: 99.3274 |  0:00:59s
epoch 3  | loss: 54882.61184| val_0_mae: 99.66273|  0:01:18s
epoch 4  | loss: 54781.98376| val_0_mae: 99.4892 |  0:01:38s
epoch 5  | loss: 54834.16983| val_0_mae: 99.36418|  0:01:57s
epoch 6  | loss: 54784.86199| val_0_mae: 99.13258|  0:02:17s
epoch 7  | loss: 54676.25836| val_0_mae: 98.90912|  0:02:36s
epoch 8  | loss: 54621.33027| val_0_mae: 98.75119|  0:02:56s
epoch 9  | loss: 54569.33982| val_0_mae: 98.64019|  0:03:16s
epoch 10 | loss: 54499.69737| val_0_mae: 98.42384|  0:03:36s
epoch 11 | loss: 54499.32717| val_0_mae: 98.35121|  0:03:56s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_mae = 97.88925


[I 2023-10-08 02:42:55,642] Trial 4 finished with value: 97.88925116629748 and parameters: {'n_d': 33, 'n_a': 8, 'n_steps': 9, 'gamma': 1.3954106897914667, 'lambda_sparse': 3.435899500927241e-05, 'lr': 1.4530706847817623e-05}. Best is trial 3 with value: 87.22914795206387.


epoch 0  | loss: 46095.39365| val_0_mae: 417.88197|  0:00:18s
epoch 1  | loss: 43015.46217| val_0_mae: 136.88145|  0:00:36s
epoch 2  | loss: 42736.49585| val_0_mae: 103.93545|  0:00:55s
epoch 3  | loss: 42568.07205| val_0_mae: 100.54177|  0:01:13s
epoch 4  | loss: 42609.04317| val_0_mae: 98.73855|  0:01:30s
epoch 5  | loss: 42485.57384| val_0_mae: 106.69002|  0:01:48s
epoch 6  | loss: 41993.5897| val_0_mae: 101.21619|  0:02:07s


KeyboardInterrupt: 